In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'fourier'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_fourier_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,321.53,-0.02,0.00,-0.01,-0.00,0.01,0.00,-0.00,0.00,-0.00,...,0.0,0.00,0.00,-0.00,-0.00,-0.0,0.00,0.00,0.0,-0.0
3,-0.02,241.28,0.00,0.00,0.01,0.01,-0.00,-0.00,0.00,-0.00,...,-0.0,-0.00,-0.00,-0.00,-0.00,-0.0,0.00,0.00,-0.0,-0.0
4,0.00,0.00,203.17,0.01,-0.00,-0.00,0.00,-0.00,0.00,0.00,...,0.0,-0.00,-0.00,-0.00,0.00,-0.0,-0.00,0.00,0.0,0.0
5,-0.01,0.00,0.01,157.73,0.01,-0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,-0.00,-0.00,-0.00,0.0,-0.00,0.00,-0.0,0.0
6,-0.00,0.01,-0.00,0.01,126.29,0.00,0.00,-0.00,-0.00,-0.00,...,-0.0,0.00,-0.00,-0.00,0.00,-0.0,-0.00,0.00,0.0,0.0
7,0.01,0.01,-0.00,-0.00,0.00,97.86,0.00,-0.00,0.00,0.00,...,-0.0,-0.00,-0.00,-0.00,0.00,0.0,-0.00,-0.00,-0.0,0.0
8,0.00,-0.00,0.00,0.00,0.00,0.00,78.49,0.00,0.00,-0.00,...,-0.0,0.00,0.00,0.00,-0.00,-0.0,-0.00,0.00,-0.0,-0.0
9,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,63.09,-0.00,0.00,...,0.0,-0.00,-0.00,-0.00,-0.00,-0.0,-0.00,0.00,0.0,-0.0
10,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,50.17,0.00,...,0.0,-0.00,0.00,-0.00,0.00,-0.0,0.00,-0.00,0.0,0.0
11,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,40.04,...,0.0,-0.00,0.00,-0.00,-0.00,0.0,-0.00,0.00,-0.0,-0.0


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,-0.00007,0.00002,-0.00003,-0.00002,0.00003,0.00003,-0.00003,0.00002,-0.00000,...,0.00002,0.00001,0.00002,-0.00002,-0.00000,-0.00000,0.00002,0.00001,0.00002,-0.00000
3,-0.00007,1.00000,0.00000,0.00002,0.00003,0.00005,-0.00002,-0.00003,0.00003,-0.00000,...,-0.00002,-0.00002,-0.00002,-0.00001,-0.00001,-0.00000,0.00000,0.00001,-0.00000,-0.00001
4,0.00002,0.00000,1.00000,0.00007,-0.00003,-0.00001,0.00002,-0.00001,0.00001,0.00000,...,0.00000,-0.00000,-0.00001,-0.00002,0.00001,-0.00001,-0.00000,0.00001,0.00000,0.00001
5,-0.00003,0.00002,0.00007,1.00000,0.00004,-0.00002,0.00002,0.00005,0.00001,0.00000,...,0.00001,0.00002,-0.00001,-0.00002,-0.00002,0.00001,-0.00000,0.00001,-0.00000,0.00000
6,-0.00002,0.00003,-0.00003,0.00004,1.00000,0.00000,0.00003,-0.00001,-0.00002,-0.00000,...,-0.00000,0.00001,-0.00000,-0.00001,0.00001,-0.00000,-0.00000,0.00001,0.00000,0.00000
7,0.00003,0.00005,-0.00001,-0.00002,0.00000,1.00000,0.00002,-0.00000,0.00001,0.00000,...,-0.00004,-0.00000,-0.00000,-0.00000,0.00002,0.00001,-0.00003,-0.00002,-0.00001,0.00002
8,0.00003,-0.00002,0.00002,0.00002,0.00003,0.00002,1.00000,0.00007,0.00004,-0.00000,...,-0.00003,0.00000,0.00001,0.00000,-0.00004,-0.00001,-0.00000,0.00000,-0.00001,-0.00001
9,-0.00003,-0.00003,-0.00001,0.00005,-0.00001,-0.00000,0.00007,1.00000,-0.00007,0.00001,...,0.00001,-0.00004,-0.00000,-0.00001,-0.00002,-0.00001,-0.00000,0.00000,0.00001,-0.00000
10,0.00002,0.00003,0.00001,0.00001,-0.00002,0.00001,0.00004,-0.00007,1.00000,0.00005,...,0.00000,-0.00000,0.00001,-0.00003,0.00001,-0.00000,0.00001,-0.00000,0.00000,0.00000
11,-0.00000,-0.00000,0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00001,0.00005,1.00000,...,0.00001,-0.00002,0.00001,-0.00001,-0.00002,0.00001,-0.00002,0.00000,-0.00001,-0.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.04602672834278872

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.23932631e+03 1.82305400e+03 1.28756528e+03 7.75824096e+02
 4.96354728e+02 2.97929222e+02 1.91313671e+02 1.23365240e+02
 7.78995140e+01 4.94970426e+01 3.06130264e+01 2.06560393e+01
 1.14450156e+01 7.42784443e+00 4.44149709e+00 2.60684462e+00
 1.48372372e+00 8.34358758e-01 4.53086091e-01 2.51869712e-01
 1.26918299e-01 6.17979805e-02 2.90198013e-02 1.33776195e-02
 5.46919448e-03 2.07655411e-03 7.43574526e-04 2.37319898e-04
 2.97550908e-05 4.16319500e-06 3.22837641e-07 6.38837226e-28]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.997699,-0.052820,-0.031556,-0.019245,-0.013772,-0.009882,-0.007638,-0.006034,-0.004703,-0.003727,...,-0.000090,-0.000062,-0.000040,-0.000026,-0.000015,-0.000009,-0.000003,-0.000001,-3.395699e-07,-1.049962e-07
1,0.048301,0.993343,-0.093445,-0.034914,-0.021772,-0.014554,-0.010943,-0.008440,-0.006493,-0.005112,...,-0.000125,-0.000086,-0.000055,-0.000034,-0.000021,-0.000012,-0.000004,-0.000002,-5.614491e-07,-1.600279e-07
2,0.033683,0.087733,0.991999,-0.069529,-0.035118,-0.021466,-0.015458,-0.011695,-0.008915,-0.006953,...,-0.000166,-0.000114,-0.000074,-0.000046,-0.000027,-0.000016,-0.000006,-0.000002,-7.177676e-07,-1.656167e-07
3,0.021142,0.036735,0.061162,0.992730,-0.080984,-0.035242,-0.022822,-0.016319,-0.012124,-0.009286,...,-0.000215,-0.000147,-0.000096,-0.000060,-0.000036,-0.000021,-0.000008,-0.000003,-9.729407e-07,-2.337552e-07
4,0.016144,0.024805,0.034650,0.073135,0.992151,-0.074277,-0.037465,-0.024413,-0.017215,-0.012812,...,-0.000286,-0.000195,-0.000126,-0.000079,-0.000047,-0.000028,-0.000010,-0.000004,-1.235250e-06,-3.126424e-07
5,0.011766,0.016944,0.021896,0.034839,0.064928,0.991357,-0.087995,-0.041455,-0.025765,-0.018100,...,-0.000376,-0.000255,-0.000165,-0.000102,-0.000061,-0.000036,-0.000014,-0.000005,-1.694927e-06,-3.560905e-07
6,0.009553,0.013400,0.016660,0.024134,0.036521,0.077798,0.989649,-0.093532,-0.042552,-0.026847,...,-0.000493,-0.000333,-0.000215,-0.000134,-0.000083,-0.000047,-0.000017,-0.000007,-2.258225e-06,-5.560483e-07
7,0.007924,0.010881,0.013285,0.018288,0.025511,0.041652,0.081954,0.988824,-0.094324,-0.044099,...,-0.000639,-0.000442,-0.000282,-0.000175,-0.000108,-0.000062,-0.000022,-0.000009,-2.762076e-06,-7.046242e-07
8,0.006420,0.008713,0.010546,0.014169,0.018863,0.027580,0.042210,0.081672,0.988131,-0.099288,...,-0.000835,-0.000572,-0.000368,-0.000231,-0.000138,-0.000081,-0.000029,-0.000012,-3.671042e-06,-9.008578e-07
9,0.005332,0.007190,0.008631,0.011408,0.014800,0.020544,0.028637,0.043557,0.085989,0.987439,...,-0.001098,-0.000755,-0.000484,-0.000302,-0.000184,-0.000106,-0.000039,-0.000015,-4.954132e-06,-1.222943e-06


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.002300951724109046,
 0.0066573423500299045,
 0.00800149370760006,
 0.007270290991014616,
 0.007849438237448547,
 0.00864347893176276,
 0.010351484038314984,
 0.011176425641048038,
 0.011868706377780036,
 0.012560646233487294,
 0.01524039410451794,
 0.015270656806925853,
 0.014757376018106672,
 0.01733270849678803,
 0.0165273264709076,
 0.01677584938522747,
 0.017375942866676097,
 0.01824944914131088,
 0.020116984821593187,
 0.02110157423182657,
 0.020413116053253244,
 0.021573443987201624,
 0.023818585104191214,
 0.025134765110561563,
 0.025467896595829487,
 0.028375684809552215,
 0.032694810540089714,
 0.026604919610764832,
 0.020144604924996257,
 0.028541816289155753,
 0.0465855293942915,
 0.03287404910417924]